In [16]:
from sympy import *
import pandas as pd
import re

def superscriptify(expr):
    """Convert an expression's powers (**) into unicode superscripts and leave multiplication as is."""
    if expr is None:
        return "*"
    s = str(expr)

    # Map digits and minus sign to superscripts
    superscript_map = {
        '0': '⁰', '1': '¹', '2': '²', '3': '³', '4': '⁴',
        '5': '⁵', '6': '⁶', '7': '⁷', '8': '⁸', '9': '⁹',
        '-': '⁻'
    }

    def replace_power(match):
        exponent_str = match.group(1)
        # Convert each character of the exponent to its superscript equivalent
        return ''.join(superscript_map.get(ch, ch) for ch in exponent_str)

    # Replace all occurrences of '**<integer>' with the corresponding superscript characters
    s = re.sub(r'\*\*(-?\d+)', lambda m: replace_power(m), s)

    return s

def extended_euclidean_algorithm_polys(N, M, x, beatify: bool = true):
    # Initialize variables for the extended Euclidean algorithm
    r = [N, M]  # Remainders
    s = [Poly(1, x), Poly(0, x)]  # Coefficients of N(x)
    t = [Poly(0, x), Poly(1, x)]  # Coefficients of M(x)

    # Perform the extended Euclidean algorithm
    k = 0
    while r[k+1] != 0:
        quotient, remainder = div(r[k], r[k+1])
        r.append(remainder)
        if remainder == 0:
            s.append(None)
            t.append(None)
        else:
            s.append(s[k] - quotient * s[k+1])
            t.append(t[k] - quotient * t[k+1])
        k += 1

    results = []
    for i in range(len(r)):
        if beatify:
            r_str = superscriptify(r[i].as_expr() if r[i] is not None else None)
            s_str = superscriptify(s[i].as_expr() if s[i] is not None else None)
            t_str = superscriptify(t[i].as_expr() if t[i] is not None else None)
        else:
            r_str = r[i].as_expr() if r[i] is not None else None
            s_str = s[i].as_expr() if s[i] is not None else None
            t_str = t[i].as_expr() if t[i] is not None else None
        results.append({
            "k": i,
            "r_k(x)": r_str,
            "s_k(x)": s_str,
            "t_k(x)": t_str
        })

    df = pd.DataFrame(results)
    return df

# Example usage
x = symbols('x', real=True)
N = Poly(x**4 + x**3 - 2*x**2 + 3*x - 43, x)  # N(x)
M = Poly(x**2 + 2*x - 6, x)                  # M(x)
df_results = extended_euclidean_algorithm_polys(N, M, x, true)


In [17]:
df_results.head(10)

,k,r_k(x),s_k(x),t_k(x)
0,0,x⁴ + x³ - 2*x² + 3*x - 43,1,0
1,1,x² + 2*x - 6,0,1
2,2,-15*x - 7,1,-x² + x - 6
3,3,-1511/225,x/15 + 23/225,-x³/15 - 8*x²/225 - 67*x/225 + 29/75
4,4,0,*,*


# Inverse of mod

In [20]:
mod_inverse(5,6)

5

# Chinese remainder therom

In [21]:
from sympy.ntheory.modular import crt

# Define the moduli
moduli = [3, 5, 7]

# Define the remainders
remainders = [2, 3, 5]

# Solve using the Chinese Remainder Theorem
solution, modulus = crt(moduli, remainders)

# Print the solution
print(f"x ≡ {solution} (mod {modulus})")

x ≡ 68 (mod 105)


# Problem: Equivalent Sets

**conjugated** of a set. Given a universal set $U$, the **conjugated** of a set $A \subseteq U$ is the set of all elements in $U$ that are not in $A$. Formally:

$$
\overline{A} = \{ x \in U : x \notin A \}
$$


## Question:
Given a universal set $ U $, which of the following sets are equal to $ \overline{A \cap (B \cup C)} $?

### Options:
1. $ \text{None of these} $
2. $ (A \cap (B \cup C)) - U $
3. $ \overline{A} \cap (\overline{B} \cup \overline{C}) $
4. $ A \cup (B \cap C) $
5. $ (U - (A \cap B)) \cap (U - (A \cap C)) $

### Answer:
**Step 1: Start with the given expression**

$$
\overline{A \cap (B \cup C)}
$$

**Step 2: Apply De Morgan's Laws**

Using De Morgan's law:
$$
\overline{X \cap Y} = \overline{X} \cup \overline{Y}
$$
we can rewrite:
$$
\overline{A \cap (B \cup C)} = \overline{A} \cup \overline{B \cup C}
$$

Now apply De Morgan's law again to $\overline{B \cup C}$:
$$
\overline{B \cup C} = \overline{B} \cap \overline{C}
$$

So we have:
$$
\overline{A \cap (B \cup C)} = \overline{A} \cup (\overline{B} \cap \overline{C})
$$

This is our simplified form.

**Step 3: Compare with the given options**

1. **None of these:** We should check all other options first.

2. $(A \cap (B \cup C)) - U$:

   Subtracting the universal set $U$ from any set results in the empty set. Thus:
   $$
   (A \cap (B \cup C)) - U = \emptyset
   $$
   This clearly doesn't match $\overline{A \cap (B \cup C)}$.

3. $\overline{A} \cap (\overline{B} \cup \overline{C})$:

   Our derived form is $\overline{A} \cup (\overline{B} \cap \overline{C})$. Notice that:
   $$
   \overline{A} \cup (\overline{B} \cap \overline{C}) \neq \overline{A} \cap (\overline{B} \cup \overline{C})
   $$
   They are not the same due to the difference in how unions and intersections distribute.

4. $A \cup (B \cap C)$:

   This doesn't match the complement form we have. Substituting a few test cases quickly shows it doesn't align with $\overline{A \cap (B \cup C)}$.

5. $(U - (A \cap B)) \cap (U - (A \cap C))$:

   Let's rewrite this using complements. Since $U - X = \overline{X}$, we get:
   $$
   (U - (A \cap B)) \cap (U - (A \cap C)) = \overline{A \cap B} \cap \overline{A \cap C}
   $$

   Apply De Morgan's law to each:
   $$
   \overline{A \cap B} = \overline{A} \cup \overline{B}
   $$
   $$
   \overline{A \cap C} = \overline{A} \cup \overline{C}
   $$

   Substitute these back:
   $$
   (\overline{A} \cup \overline{B}) \cap (\overline{A} \cup \overline{C})
   $$

   Now use the distributive law:
   $$
   (\overline{A} \cup \overline{B}) \cap (\overline{A} \cup \overline{C}) = \overline{A} \cup (\overline{B} \cap \overline{C})
   $$

   This matches exactly with our simplified form of $\overline{A \cap (B \cup C)}$.

**Conclusion:**

Option 5 is exactly equal to $\overline{A \cap (B \cup C)}$.

---

# Final Answer

$$
\boxed{(U - (A \cap B)) \cap (U - (A \cap C))}
$$

Thus, the correct choice is **Option 5**.

# Permutations

Consider all permutations of abcde.
1. How many do not contain abc?
2. How many contain ab or bc?
3. How many contain ab or bc but not abc?

In [22]:
from itertools import permutations

# All permutations of 'abcde'
p = permutations('abcde')
p_list = list(p)

# Filter permutations not containing 'abc'
no_abc = [perm for perm in p_list if 'abc' not in ''.join(perm)]
print(len(no_abc))

# Filter permutations containing 'ab' or 'bc'
ab_bc = [perm for perm in p_list if 'ab' in ''.join(perm) or 'bc' in ''.join(perm)]
print(len(ab_bc))

# Filter permutations containing 'ab' or 'bc' but not 'abc'
ab_bc_no_abc = [perm for perm in ab_bc if 'abc' not in ''.join(perm)]
print(len(ab_bc_no_abc))


114
42
36


# Coefficients

The coefficient of $x^7 \cdot y^2$ in $(2x+3y)^9$ is:

In [23]:
# Define the variables
x, y = symbols('x y')

# Define the expression
expr = (2*x + 3*y)**9

# Expand the expression
expanded_expr = expand(expr)

# Extract the coefficient of x^7 * y^2
coefficient = expanded_expr.coeff(x**7 * y**2)

# Factorize the coefficient
factorized_coefficient = factorint(coefficient)
factorized_coefficient


{2: 9, 3: 4}

## If it doesn't work use Binomial Theorem (Manual Approach):

### General Term in the Expansion of $ (2x + 3y)^9 $

The general term in the expansion of $ (2x + 3y)^9 $ is given by:

$$
T_k = \binom{9}{k} \cdot (2x)^{9-k} \cdot (3y)^k
$$

---

For $ x^7 \cdot y^2 $, we have:

$$
9 - k = 7 \implies k = 2
$$

---

### Substitute $ k = 2 $:

The coefficient of $ x^7 \cdot y^2 $ is:

$$
\text{Coefficient} = \binom{9}{2} \cdot (2)^7 \cdot (3)^2
$$

### Step 2: Substitute $ k = 2 $

The coefficient of $ x^7 y^2 $ is:

$$
\text{Coefficient} = \binom{9}{2} \cdot (2)^7 \cdot (3)^2
$$

---

### Step 3: Compute Each Part

1. **Binomial Coefficient:**
   $$
   \binom{9}{2} = \frac{9 \cdot 8}{2} = 36
   $$

2. **Power of 2:**
   $$
   (2)^7 = 128
   $$

3. **Power of 3:**
   $$
   (3)^2 = 9
   $$

4. **Combine All:**
   $$
   \text{Coefficient} = 36 \cdot 128 \cdot 9
   $$

---

### Step 4: Simplify the Expression

Simplify step-by-step:

1. Multiply:
   $$
   36 \cdot 128 = 4608
   $$
2. Multiply further:
   $$
   4608 \cdot 9 = 41472
   $$

This simplifies in terms of powers of 2 and 3:
$$
41472 = 2^9 \cdot 3^4
$$



# Number of derangements
Let $ D_n $ denote the number of derangements of $ n $ objects. For instance, $ D_3 = 2 $, because the derangements of $ 123 $ are $ 231 $ and $ 312 $. We will evaluate $ D_n $ for all positive integers $ n $ using the principle of inclusion–exclusion.

---

## **Theorem 2**

The number of derangements of a set with $ n $ elements is:

$$
D_n = n! \left[ 1 - \frac{1}{1!} + \frac{1}{2!} - \frac{1}{3!} + \cdots + (-1)^n \frac{1}{n!} \right].
$$

In [24]:
import math
def derangement_formula(n):
    """ Calculate the number of derangements using the direct formula. """
    derangements = math.factorial(n) * sum(((-1) ** k) / math.factorial(k) for k in range(n + 1))
    return int(derangements)

In [25]:
derangement_formula(6)

265

# Cube graph

In [26]:
def count_edges_in_n_cube(n):
    # Number of edges in the n-cube graph
    return n * (2 ** (n - 1))

# Example: Edges in a 3-cube
n = 4
print(f"Number of edges in a {n}-cube: {count_edges_in_n_cube(n)}")


Number of edges in a 4-cube: 32


# Min number sum (pigeonhole principle)

In [27]:
from itertools import combinations

def guaranteed_selection_count(target, numbers):
    # Step 1: Find all subsets whose sum equals the target
    target_subsets = []
    for r in range(1, len(numbers) + 1):
        for subset in combinations(numbers, r):
            if sum(subset) == target:
                target_subsets.append(set(subset))

    # If no subset sums to the target, no finite number will guarantee it.
    if not target_subsets:
        return float('inf')  # or len(numbers) + 1, depending on your preference

    # Step 2: Find the maximum subset of 'numbers' that does NOT contain any target-sum subset fully
    # We'll brute force all subsets of 'numbers' and check.
    # For each candidate subset, we must ensure it does not include any of the target_subsets fully.
    # i.e., For every target_subset T, candidate_subset should not be a superset of T.

    max_safe_size = 0
    n = len(numbers)
    for r in range(n + 1):
        for candidate in combinations(numbers, r):
            candidate_set = set(candidate)
            # Check if candidate_set fully contains any target_subset
            if any(t_subset.issubset(candidate_set) for t_subset in target_subsets):
                # This candidate is not safe because it contains a full target-sum subset
                continue
            # If it passes all checks, update max_safe_size
            max_safe_size = max(max_safe_size, r)

    # Step 3: Once you exceed max_safe_size by 1, you guarantee forming a target subset
    return max_safe_size + 1


In [28]:
# Example usage:
numbers = [1, 3, 5, 7, 9, 11, 13, 15]
target = 16
count = guaranteed_selection_count(target, numbers)
print(f"Minimum number of selections needed to guarantee a sum of {target}: {count}")

Minimum number of selections needed to guarantee a sum of 16: 5


In [29]:
# Example usage:
numbers = [1, 2, 3, 4, 5, 6]
target = 7
count = guaranteed_selection_count(target, numbers)
print(f"Minimum number of selections needed to guarantee a sum of {target}: {count}")

Minimum number of selections needed to guarantee a sum of 7: 4


# Relation
**Reflexive:** A binary relation $ R $ on a set $ A $ is called *reflexive* if every element in $ A $ is related to itself. Formally, for every $ a \in A $,
$$
(a, a) \in R.
$$

**Symmetric:** A binary relation $ R $ on a set $ A $ is *symmetric* if whenever $ a $ is related to $ b $, then $ b $ is also related to $ a $. Formally, for every $ a, b \in A $,
$$
(a, b) \in R \implies (b, a) \in R.
$$

**Antisymmetric:** A binary relation $ R $ on a set $ A $ is *antisymmetric* if the only way for both $ a $ to be related to $ b $ and $ b $ to be related to $ a $ to hold simultaneously is when $ a = b $. Formally, for every $ a, b \in A $,
$$
(a, b) \in R \text{ and } (b, a) \in R \implies a = b.
$$

**Transitive:** A binary relation $ R $ on a set $ A $ is *transitive* if whenever $ a $ is related to $ b $ and $ b $ is related to $ c $, then $ a $ must also be related to $ c $. Formally, for every $ a, b, c \in A $,
$$
(a, b) \in R \text{ and } (b, c) \in R \implies (a, c) \in R.
$$


# Sets and subset sizes

In [ ]:
from sympy import binomial, S

# Total set size
n = 80

# Question 1: Subsets of size exactly 20
subsets_size_20 = binomial(n, 20)

# Question 2: Subsets with no element greater than 60
subsets_no_greater_than_60 = 2**60  # 60 elements can form all possible subsets

# Question 3: Subsets with an even number of elements
subsets_even_elements = sum(binomial(n, k) for k in range(0, n + 1, 2))

subsets_size_20, subsets_no_greater_than_60, subsets_even_elements


(3535316142212174320, 1152921504606846976, 604462909807314587353088)

604462909807314587353088 = $2^{79}$